In [1]:
# !pwd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

# checkpoint_path = "KoBART-summarization/checkpoint/model_chp/epoch=02-val_loss=2.316.ckpt"
# checkpoint_path = "KoBART-summarization/checkpoint/last-v1.ckpt"
# checkpoint_path = "KoBART-summarization/checkpoint/last-v2.ckpt"
checkpoint_path = "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/last-v3.ckpt"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [4]:
# 모델 가중치 로드
checkpoint = torch.load(checkpoint_path)
checkpoint = dict(checkpoint['state_dict'])

def remove_model_prefix(dictionary):
    new_dict = {}
    for key, value in dictionary.items():
        if key.startswith("model."):
            new_key = key[len("model."):]  # "model." 부분을 제거합니다.
            new_dict[new_key] = value
        else:
            new_dict[key] = value
    return new_dict

new_data = remove_model_prefix(checkpoint)


# model.load_state_dict(dict(checkpoint['state_dict']))
model.load_state_dict(new_data)

<All keys matched successfully>

In [5]:
# text = """
# 1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
# 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
# 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
# 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
# 확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
# 오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
# 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
# 비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
# 부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
# 집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
# 한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
# 이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
# """

In [6]:
def summarize(text):
    text = text.replace('\n', ' ')
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=2,  max_length= 1000,  early_stopping=True) # eos_token_id=0,
    result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    return result

# def summarize(text):
#     text = text.replace('\n', ' ')
#     # 주어진 텍스트에 있는 단어만 필터링하여 사용합니다.
#     tokens = text.split()
#     valid_tokens = [token for token in tokens if token in tokenizer.get_vocab()]
#     filtered_text = ' '.join(valid_tokens)

#     raw_input_ids = tokenizer.encode(filtered_text)
#     input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

#     summary_ids = model.generate(torch.tensor([input_ids]), num_beams=3, max_length=300, early_stopping=True)
#     result = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return result

In [7]:
# summarize(text)
# 1일 0 9시까지 최소 20만3220명이 코로나19에 신규 확진되어 역대 최다 기록을 갈아치웠다.
# '1일 0시부터 오후 9시까지 최소 17만8603명이 신규 확진되어 동시간대 최다 기록인 역대 최다 기록도 갈아치웠다.'
# 1일 0시부터 오후 9시까지 전국 신규 확진자 수가 20만3220명으로 집계되었는데 이는 동시간대 최다 기록이다.'

In [8]:
import pandas as pd
ex_list = ["/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-05_13_40.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-05_13_38.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-05_13_33.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-06_09_57.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-06_11_31.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-06_20_10.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-07_16_52.xlsx",
           "/content/drive/MyDrive/2.서울_빅데이터 분석(230907~240306)/0.프로젝트4_마지막프로젝트_240226/2.koBart(seq2seq)_sum/news_2024-03-08_10_44.xlsx",
          ]

df = pd.read_excel(ex_list[4])
display(df[:3])


,Media,Category,Title,Link,Description,Publication Date
0,연합뉴스,최신,"PSG·뮌헨, 챔스 8강행…이강인 도움·김민재 결장",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"PSG·뮌헨, 챔스 8강행…이강인 도움·김민재 결장이강인이 유럽 챔피언스리그 16강...",2024-03-06
1,연합뉴스,최신,"'전공의 집단사직' 관련 의협 간부 첫 소환 조사…""숨길 것 없어""",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"'전공의 집단사직' 관련 의협 간부 첫 소환 조사…""숨길 것 없어""[앵커]의료법 위...",2024-03-06
2,연합뉴스,최신,탈당·무소속 출마…여야 공천 막바지 후폭풍,https://www.yonhapnewstv.co.kr/news/MYH2024030...,탈당·무소속 출마…여야 공천 막바지 후폭풍[앵커]국회 상황 보겠습니다.민주당에선 '...,2024-03-06


In [9]:
a = df['Description'][133]
display(a)
print('\n 요약모델 요약내용')
summarize(a)

' 미국에서 온실가스 함유 물질의 수입을 규제하는 법이 제정된 이래 처음으로 이 법을 위반해 기소되는 사례가 나왔습니다.5일(현지시간) 미 법무부에 따르면 캘리포니아 남부지방검찰청은 전날 샌디에이고에 사는 남성 마이클 하트(58)를 온실가스 밀수 혐의로 체포한 뒤 기소했습니다.하트는 멕시코에서 온실가스 물질인 수소불화탄소(HFC)가 포함된 냉매를 멕시코에서 구매한 뒤 방수포 등으로 숨겨 자신의 차량에 싣고 미국으로 들어온 혐의를 받습니다.그는 밀반입한 냉매를 소셜미디어의 온라인 장터 등을 통해 판매해 수익을 올린 것으로 조사됐습니다.검찰청은 2020년 HFC 수입을 금지하는 법안이 시행된 이래 이 법을 적용해 기소하는 첫 사례라고 설명했습니다.미 환경보호청(EPA)에 따르면 HFC는 기후 변화를 일으키는 강력한 온실가스 물질로, 냉장고와 에어컨, 건물 단열재 등에 사용됩니다.세계 각국은 2016년에 HFC를 단계적으로 퇴출하기로 합의한 바 있으며, 미국은 2020년부터 당국의 허가 없이 HFC를 포함한 냉매를 수입하는 행위를 금지하고 있습니다.타라 맥그래스 검사는 "이번 기소는 중요한 이정표가 될 것"이라며 "우리는 독성 오염 물질로부터 지구를 보호하기 위해 가능한 모든 수단을 쓰고 있다"고 말했습니다.'


 요약모델 요약내용


'캘리포니아 남부지방검찰청은 멕시코에서 온실가스 물질인 수소불화탄소(HFC)가 포함된 냉매를 멕시코에서 구매한 뒤 방수포 등으로 숨겨 자신의 차량에 싣고 미국으로 들어온 혐의를 받는 남성 마이클 하트(58)를 온실가스 밀수 혐의로 체포한 뒤 기소했다.'

In [10]:
# a =  df[df['Description'].str.contains('전기차')]
# txts = ""
# for txt in a['Description']:
#   txts += summarize(txt)

# display(txts)
# print('\n')
# summarize(txts)

,Media,Category,Title,Link,Description,Publication Date
0,연합뉴스,최신,"PSG·뮌헨, 챔스 8강행…이강인 도움·김민재 결장",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"PSG·뮌헨, 챔스 8강행…이강인 도움·김민재 결장이강인이 유럽 챔피언스리그 16강...",2024-03-06
1,연합뉴스,최신,"'전공의 집단사직' 관련 의협 간부 첫 소환 조사…""숨길 것 없어""",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"'전공의 집단사직' 관련 의협 간부 첫 소환 조사…""숨길 것 없어""[앵커]의료법 위...",2024-03-06
2,연합뉴스,최신,탈당·무소속 출마…여야 공천 막바지 후폭풍,https://www.yonhapnewstv.co.kr/news/MYH2024030...,탈당·무소속 출마…여야 공천 막바지 후폭풍[앵커]국회 상황 보겠습니다.민주당에선 '...,2024-03-06
3,연합뉴스,최신,"[속보] 홍영표, 민주당 탈당…""이재명 사당화 분노""",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"[속보] 홍영표, 민주당 탈당…""이재명 사당화 분노""연합뉴스TV 기사문의 및 제보 ...",2024-03-06
4,연합뉴스,최신,"[뉴스포커스] '서초을 도전' 신동욱…""품격 있는 정치 이끌 것""",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"[뉴스포커스] '서초을 도전' 신동욱…""품격 있는 정치 이끌 것""총선을 35일 앞두...",2024-03-06
5,연합뉴스,최신,2월 소비자물가 3.1%↑…한 달 만에 3%대,https://www.yonhapnewstv.co.kr/news/MYH2024030...,2월 소비자물가 3.1%↑…한 달 만에 3%대[앵커]농산물값 고공행진에 국제유가 상...,2024-03-06
6,연합뉴스,최신,[현장연결] 주수호 의협 비대위 언론홍보위원장 경찰 출석,https://www.yonhapnewstv.co.kr/news/MYH2024030...,"[현장연결] 주수호 의협 비대위 언론홍보위원장 경찰 출석 ""의사들 저항은 가짜뉴스·...",2024-03-06
7,연합뉴스,최신,"전공의 7천명 미복귀 확인…정부, 주동자 고발 검토",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"전공의 7천명 미복귀 확인…정부, 주동자 고발 검토[앵커]정부가 미복귀 전공의들을 ...",2024-03-06
8,연합뉴스,최신,미 슈퍼화요일 바이든-트럼프 재격돌 확실…본선 서막,https://www.yonhapnewstv.co.kr/news/MYH2024030...,미 슈퍼화요일 바이든-트럼프 재격돌 확실…본선 서막[앵커]미국 대선 후보의 윤곽을 ...,2024-03-06
9,연합뉴스,최신,"[날씨] 흐리고 곳곳 비·눈…남해안, 제주 강풍",https://www.yonhapnewstv.co.kr/news/MYH2024030...,"[날씨] 흐리고 곳곳 비·눈…남해안, 제주 강풍오늘 전국 하늘빛은 대체로 흐리겠고,...",2024-03-06


In [44]:
df = pd.read_excel(ex_list[4])
df['Description'] = df['Description']+df['Title']
# display(df[:10])

query = "사건사고 발생한 이력이 있나"

def simility(query, news):
    intersection_cardinality = len(set.intersection(*[set(query), set(news)]))
    union_cardinality = len(set.union(*[set(query), set(news)]))
    similar = intersection_cardinality / float(union_cardinality)
    return similar
df['similar'] = df['Description'].apply(lambda news: simility(query,news))

af = df.copy()
af = af.sort_values(by='similar', ascending=False)
af = af.reset_index(drop=True)
af.head()


,Media,Category,Title,Link,Description,Publication Date,similar
0,SBS,정치,수원서 태권도장 승합차 상가 돌진…8명 부상,https://news.sbs.co.kr/news/endPage.do?news_id...,오늘 오후 4시 반쯤 경기 수원시의 한 이면도로 사거리에서 태권도 도장 원생용 승합...,2024-03-05,0.088889
1,뉴시스,속보,"[속보] 트럼프, 앨라배마 경선 승리",http://www.newsis.com/view/?id=NISX20240306_00...,"후속기사가 이어집니다[속보] 트럼프, 앨라배마 경선 승리",2024-03-06,0.088235
2,뉴시스,속보,"[속보] 트럼프, 메인·아칸소·텍사스 경선서 승리",http://www.newsis.com/view/?id=NISX20240306_00...,"후속기사가 이어집니다[속보] 트럼프, 메인·아칸소·텍사스 경선서 승리",2024-03-06,0.076923
3,SBS,사회,수원 장안구 아파트 단지 세탁소 화재…대응 1단계 발령,https://news.sbs.co.kr/news/endPage.do?news_id...,오늘 오전 10시쯤 경기 수원 장안구의 한 아파트 단지 상가 세탁소에서 불이 났습니...,2024-03-06,0.074766
4,SBS,경제,"농협은행서 109억 원 규모 배임 사고…""경찰 수사 중""",https://news.sbs.co.kr/news/endPage.do?news_id...,NH농협은행에서 109억 원 규모의 금융사고가 발생했습니다.농협은행은 업무상 배임...,2024-03-06,0.072072


In [45]:
txts= ""
for txt in range(len(af[['Description']][:6])):
  ttx = str(af.Description[txt])
  txts += summarize(ttx)
display(txts)

'오늘(5일) 오후 4시 반쯤 경기 수원시의 한 이면도로 사거리에서 태권도 도장 원생용 승합차가 승용차와 부딪친 뒤 인근 상가 건물로 돌진해 어린이 5명과 휴대전화 판매점 직원 등 총 8명이 다쳐 병원으로 이송됐는데 생명에 지장은 없는 상태인 것으로 전해졌습니다.후속기사가 이어집니다 후속기사가 이어집니다후속기사가 이어집니다 후속기사가 이어집니다오늘 오전 10시쯤 경기 수원 장안구의 한 아파트 단지 상가 세탁소에서 불이 나서,경기소방본부는 오전 10시 17분 대응 1단계를 발령하고 진화 작업에 나섰습니다.지난 5일 3월부터 지난해 11월까지 NH농협은행에서 109억 원 규모의 금융사고가 발생했습니다.오늘 오전 8시 13분쯤 전남 광양 국가산업단지 내 이차전지 양극재 생산 공장에서 배관이 파손돼 수산화리튬이 소량 유출됐다는 신고가 119 등에 접수됐는데,현장 작업자 700여 명은 곧바로 대피해 인명 피해는 없는 것으로 파악됐다.'

In [46]:
summarize(txts)

'지난 5일 NH농협은행에서 109억 원 규모의 금융사고가 발생했고,현장 작업자 700여명은 곧바로 대피해 인명 피해는 없는 것으로 파악됐다.'

# 문장유사도

## 코사인 유사도(Cosine Similarity)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
sentences = ("그건 아닌데 어떤 이상한 놈 때문에 기분 다 잡쳤어",
"아닌데 어떤 이상한 놈 때문에 기분 다 잡쳤어")

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
print("코사인 유사도 측정")
print(cos_similar[0][0])

# 코사인 유사도 측정
# 0.8673636849609287

코사인 유사도 측정
0.8673636849609287


## 2. 자카드 유사도(Jaccard Similarity)

In [15]:
answer_string = "그건 아닌데 어떤 이상한 놈 때문에 기분 다 잡쳤어"
input_string = "아닌데 어떤 이상한 놈 때문에 기분 다 잡쳤어"

intersection_cardinality = len(set.intersection(*[set(answer_string), set(input_string)]))
union_cardinality = len(set.union(*[set(answer_string), set(input_string)]))
similar = intersection_cardinality / float(union_cardinality)

print(similar)
# 0.9

0.9
